In [ ]:
! pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 7.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sn

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split, GroupShuffleSplit 

import glob
import sys
import os
import math
import gc
import sys
import sklearn
import scipy

print(f'Tensorflow V{tf.__version__}')
print(f'Keras V{tf.keras.__version__}')
print(f'Python V{sys.version}')

/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Tensorflow V2.12.0
Keras V2.12.0
Python V3.9.16 (main, Dec  7 2022, 01:11:51) 
[GCC 9.4.0]


# Config

In [ ]:
# If True, processing data from scratch
# If False, loads preprocessed data
PREPROCESS_DATA = False
TRAIN_MODEL = True
# True: use 10% of participants as validation set
# False: use all data for training -> gives better LB result
USE_VAL = False

N_ROWS = 543
N_DIMS = 2
DIM_NAMES = ['x', 'y']
SEED = 2023
NUM_CLASSES = 250

INPUT_SIZE = 64
N_COLS = 21 # dominant side index

BATCH_ALL_SIGNS_N = 4
BATCH_SIZE = 256
N_EPOCHS = 100
LR_MAX = 1e-3
N_WARMUP_EPOCHS = 0
WD_RATIO = 0.05
MASK_VAL = 4237

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1 Baseline Model

In [ ]:
%cd /content/drive/MyDrive/NUS_MSBA/BT5153/group project/baseline model inputs

/content/drive/MyDrive/NUS_MSBA/BT5153/group project/baseline model inputs


In [ ]:
X_train = np.load("X_train.npy")
y_train = np.load("y_train.npy")

X_val = np.load("X_val.npy")
y_val = np.load("y_val.npy")

In [ ]:
num_classes = 250
y_train_one_hot = np.zeros((y_train.shape[0], num_classes))
y_train_one_hot[np.arange(y_train.shape[0]), y_train] = 1

In [ ]:
y_val_one_hot = np.zeros((y_val.shape[0], num_classes))
y_val_one_hot[np.arange(y_val.shape[0]), y_val] = 1

In [ ]:
import tensorflow as tf

batch_size = 1000

# Define the input shape
input_shape = (64, 21, 2)

# Define the number of classes
num_classes = 250

# Define the model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPoolingD(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 62, 19, 32)        608       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 9, 32)        0         
 )                                                               
                                                                 
 conv2d_3 (Conv2D)           (None, 29, 7, 64)         18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 3, 64)        0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 2688)              0         
                                                                 
 dense_7 (Dense)             (None, 128)              

In [ ]:
num_epochs = 30

# Create the dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train_one_hot)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val_one_hot)).batch(batch_size)

# Create the variables outside of the tf.function decorator
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        preds = model(x)
        loss = loss_fn(y, preds)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

# Train the model
for epoch in range(num_epochs):
    for x_batch, y_batch in train_dataset:
        loss = train_step(x_batch, y_batch)

    # Compute validation loss and accuracy
    val_loss, val_acc = model.evaluate(val_dataset)

    print(f'Epoch {epoch+1}, Loss: {loss:.3f}, Val Loss: {val_loss:.3f}, Val Acc: {val_acc:.3f}')

5/5 [==============================] - 2s 394ms/step - loss: 4.7231 - accuracy: 0.0750
Epoch 1, Loss: 2.656, Val Loss: 4.723, Val Acc: 0.075
5/5 [==============================] - 2s 398ms/step - loss: 4.7776 - accuracy: 0.0744
Epoch 2, Loss: 2.050, Val Loss: 4.778, Val Acc: 0.074
5/5 [==============================] - 2s 396ms/step - loss: 4.7788 - accuracy: 0.0748
Epoch 3, Loss: 1.970, Val Loss: 4.779, Val Acc: 0.075
5/5 [==============================] - 2s 391ms/step - loss: 4.7675 - accuracy: 0.0800
Epoch 4, Loss: 1.813, Val Loss: 4.767, Val Acc: 0.080
5/5 [==============================] - 2s 402ms/step - loss: 4.7652 - accuracy: 0.0808
Epoch 5, Loss: 1.729, Val Loss: 4.765, Val Acc: 0.081
5/5 [==============================] - 2s 395ms/step - loss: 4.7572 - accuracy: 0.0849
Epoch 6, Loss: 1.626, Val Loss: 4.757, Val Acc: 0.085
5/5 [==============================] - 2s 402ms/step - loss: 4.7538 - accuracy: 0.0872
Epoch 7, Loss: 1.559, Val Loss: 4.754, Val Acc: 0.087
5/5 [========

In [ ]:
# Save Model Weights
model.save_weights('baseline_model.h5')